# Scraping DailyMail specific articles

In [1]:
import pandas as pd
import numpy as np
import urllib2
import requests
import lxml
import re
from unidecode import unidecode

import time

try:
    import scrapy
except ImportError:
    import pip
    pip.main(['install', 'scrapy'])
    import scrapy
    
#from BeautifulSoup import BeautifulSoup
from bs4 import BeautifulSoup

#try:
#    import newspaper
#except ImportError:
#    import pip
#    pip.main(['install', 'newspaper'])
#    import newspaper

In [2]:
fb_data = pd.read_csv('DailyMail_facebook_statuses.csv')

In [3]:
#fb_data.head()

In [4]:
fb_data = fb_data[fb_data['status_link'].str.contains("http://dailym.ai/")]

In [5]:
#fb_data

In [6]:
links = fb_data['status_link']

In [7]:
links.to_csv('DailyMail_facebook_statuses_links.csv')

In [8]:
links.head()


0    http://dailym.ai/2dIAo6o
1    http://dailym.ai/2egmHRz
2    http://dailym.ai/2e1bAvW
3    http://dailym.ai/2exVKbs
4    http://dailym.ai/2dIqhic
Name: status_link, dtype: object

In [9]:
links[0]

'http://dailym.ai/2dIAo6o'

In [10]:
def get_body(url):
    page = urllib2.urlopen(url).read().decode('utf8')
    soup = BeautifulSoup(page)
    html = soup.prettify()
    #text = ' '.join(map(lambda p: p.text, soup.find_all('p')))
    #return soup.title.text, text
    return soup.title.text, soup.content

In [11]:
def get_text(url):
    """ return title, description and text from the body of a given html """
    try:
        response = requests.get(url, timeout=20)
        response.encoding = 'ISO-88959-1'

        #Create a bs-parser class
        soup = BeautifulSoup(response.content, 'lxml')


        #Try to catch title and description tag
        title = re.sub(' \| Daily Mail Online', '', unidecode(soup.title.string))

        tmpall = soup.findAll('meta')
        for meta in tmpall:
            if 'description' == meta.get('name', '').lower():
                description = meta['content']
                break

        #Remove certain tags from html
        for script in soup(["script", "style", "img", "video"]):
            script.extract()

        tmpall = soup.findAll("div", {"itemprop" : "articleBody"})

        text = unidecode(' '.join(map( lambda p: p.text, soup.find_all("p", {"class" : "mol-para-with-font"}))))

        return title, description, text
    except Exception as err:
        print(err)
        return '', '', ''

In [12]:
title, descr, text = get_text(links[0])

print("Title")
print(title)
print
print("Description")
print(descr)
print
print("Text")
print(text)

Title
Manchester United stars Paul Pogba, Wayne Rooney and Zlatan Ibrahimovic warm up for Liverpool clash with stretching session in car park and city stroll

Description
Manchester United geared up for their clash with old foes Liverpool in a car park as the whole of the footballing world focuses on the titanic clash at Anfield on Monday night.

Text
Manchester United geared up for their bout with old foes Liverpool in a car park as the whole of the footballing world focuses on the titanic clash at Anfield on Monday night. The Red Devils prepared for their match against Jurgen Klopp's men with a stretching session close to the Titanic Hotel and sought peace of mind before the much-hyped bout with a stroll around the city centre. Wayne Rooney has been heavily scrutinised for his performances this season but looked happy to be back in Liverpool as he stretched with his United team-mates. Eric Bailly has been a commanding figure in the heart of the Red Devils' defence since joining from 

Displaying the HTML that we catched...

In [16]:
for url in reversed(links):
    title, descr, text = get_text(url)
    with open("DailyMail_titles.txt", "a") as myfile:
        myfile.write(title.encode('utf-8') + '. ')
    with open("DailyMail_descriptions.txt", "a") as myfile:
        myfile.write(descr.encode('utf-8') + ' <br /> ')
    with open("DailyMail_content.txt", "a") as myfile:
        myfile.write(text.encode('utf-8') + '<br /> ')
    #time.sleep(0.2)
    print(url)

http://dailym.ai/2aZ9rif
http://dailym.ai/2aWF23R
http://dailym.ai/2anzL4J
http://dailym.ai/2aYRaSd
http://dailym.ai/2aOf7sC
http://dailym.ai/2aNAuKB
http://dailym.ai/2aDDA4M
http://dailym.ai/2as2TDi
http://dailym.ai/2arMRto
http://dailym.ai/2aX0i9J
http://dailym.ai/2aDFrq0
http://dailym.ai/2b1zphR
http://dailym.ai/2aZzgyR
http://dailym.ai/2aOt2Pe
http://dailym.ai/2asaotZ
http://dailym.ai/2aWJ0JW
http://dailym.ai/2aONKOY
http://dailym.ai/2b8YjM4
http://dailym.ai/2b1i3lb
http://dailym.ai/2aWVyAL
http://dailym.ai/2aOCFh3
http://dailym.ai/2aZc6bP
http://dailym.ai/2aoxR3y
http://dailym.ai/2b1WXmR
http://dailym.ai/2ar6cKS
http://dailym.ai/2b83aNw
http://dailym.ai/2b8Zgng
http://dailym.ai/2ar24uC
http://dailym.ai/2b80LlC
http://dailym.ai/2aW7brM
http://dailym.ai/2aoDpuW
http://dailym.ai/2aXKRhA
http://dailym.ai/2aBtZhi
http://dailym.ai/2asOWoO
http://dailym.ai/2aAI36i
http://dailym.ai/2aWVhOl
http://dailym.ai/2aALSZy
http://dailym.ai/2b80oYt
http://dailym.ai/2aAPfzy
http://dailym.ai/2aWsnh8


KeyboardInterrupt: 

In [43]:
#from IPython.core.display import display, HTML
#display(HTML(str(text)))

In [45]:
#title, body = get_body(links[0])

In [47]:
#print(title)

In [48]:
#print(body)